<a href="https://colab.research.google.com/github/austinpatrickm/Metaphor-Generation/blob/main/metaphor_120821.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 12.0 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.8 MB/s 


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import torch

In [4]:
transformer_model = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(transformer_model)
model = AutoModelForSequenceClassification.from_pretrained(transformer_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
!wget https://raw.githubusercontent.com/RuiMao1988/Sequential-Metaphor-Identification/master/data/VUAsequence/VUA_seq_formatted_train.csv
!wget https://raw.githubusercontent.com/RuiMao1988/Sequential-Metaphor-Identification/master/data/VUAsequence/VUA_seq_formatted_val.csv
!wget https://raw.githubusercontent.com/RuiMao1988/Sequential-Metaphor-Identification/master/data/VUAsequence/VUA_seq_formatted_test.csv

--2022-08-21 15:17:15--  https://raw.githubusercontent.com/RuiMao1988/Sequential-Metaphor-Identification/master/data/VUAsequence/VUA_seq_formatted_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2736335 (2.6M) [text/plain]
Saving to: ‘VUA_seq_formatted_train.csv’

VUA_seq_formatted_t 100%[===================>]   2.61M  --.-KB/s    in 0.01s   

2022-08-21 15:17:15 (261 MB/s) - ‘VUA_seq_formatted_train.csv’ saved [2736335/2736335]

--2022-08-21 15:17:15--  https://raw.githubusercontent.com/RuiMao1988/Sequential-Metaphor-Identification/master/data/VUAsequence/VUA_seq_formatted_val.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.gi

In [6]:
df_train = pd.read_csv('VUA_seq_formatted_train.csv', encoding = "ISO-8859-1")
df_val = pd.read_csv('VUA_seq_formatted_val.csv', encoding = "ISO-8859-1")
df_test = pd.read_csv('VUA_seq_formatted_test.csv', encoding = "ISO-8859-1")

Load sentences, metaphor labels (0 = literal, 1 = metaphor), and position index of the adjective

In [ ]:
def loadData(df):

  # function to adapt position based on number of subtokens for each word
  def tokenizedPositionIndex(sentence, position):
    s = sentence.split(' ')
    new_position = position + 1 # cls token
    for w in s[:position]:
      tokens = tokenizer.encode(w, add_special_tokens=False)
      # if word consists of subtokens, we need to increase the index
      if len(tokens) > 1:
        new_position += len(tokens) - 1
    return new_position

  sentences = []
  labels = []
  positions = []

  for index, row in df.iterrows():
    sentence = row['sentence']
    label_seq = json.loads(row['label_seq'])
    pos_seq = row['pos_seq'].replace("'", '"')
    pos_seq = json.loads(pos_seq)
    
    for i, (s,l,p) in enumerate(zip(sentence, label_seq, pos_seq)):
      if p == 'ADJ':
        sentences.append(sentence)
        labels.append(label_seq[i])
        positions.append(i)

  newPositions = []
  for s, p in zip(sentences, positions):
    newPositions.append(tokenizedPositionIndex(s,p))
  
  return sentences, newPositions, labels

In [ ]:
sentences_train, positions_train, labels_train = loadData(df_train)
sentences_val, positions_val, labels_val = loadData(df_val)
sentences_test, positions_test, labels_test = loadData(df_test)

In [ ]:
encodings_train = tokenizer(sentences_train, truncation=True, padding=True, max_length=136)
encodings_val = tokenizer(sentences_val, truncation=True, padding=True, max_length=136)
encodings_test = tokenizer(sentences_test, truncation=True, padding=True, max_length=136)

class to process dataset; `token_type_ids` is all-zero vector, except for one on position of the adjective (if adjective consists of subtokens, we use the first subtoken as index position)

In [ ]:
class MetaphorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, positions, labels):
        self.encodings = encodings
        self.positions = positions
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        #token_type_ids is vector of zeros, we change the position of the adjective to 1
        item['token_type_ids'][self.positions[idx]] = 1
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
dataset_train = MetaphorDataset(encodings_train, positions_train, labels_train)
dataset_val = MetaphorDataset(encodings_val, positions_val, labels_val)
dataset_test = MetaphorDataset(encodings_test, positions_test, labels_test)


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=5e-5,
    fp16=True,
    evaluation_strategy='steps',
    eval_steps=100,
)


trainer = Trainer(
    model=model,                         # model with classification head
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_train,         # training dataset
    eval_dataset=dataset_val,             # evaluation dataset
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 9673
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 606


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.376700,0.327067,0.892001,0.049315,0.428571,0.026163
200,0.259100,0.240977,0.903206,0.297968,0.666667,0.191860
300,0.214500,0.217175,0.915344,0.517730,0.663636,0.424419
400,0.162200,0.233355,0.919079,0.575163,0.656716,0.511628
500,0.158700,0.215247,0.920012,0.607634,0.639871,0.578488
600,0.199300,0.222251,0.924370,0.597015,0.694981,0.523256


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1316: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 64
***** Running Evaluation *****
  Num examples = 3213
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3213
  Batch size =

TrainOutput(global_step=606, training_loss=0.23312793617988184, metrics={'train_runtime': 311.9222, 'train_samples_per_second': 62.022, 'train_steps_per_second': 1.943, 'total_flos': 1163178003531960.0, 'train_loss': 0.23312793617988184, 'epoch': 2.0})

In [ ]:
pred = trainer.predict(dataset_test)

***** Running Prediction *****
  Num examples = 3965
  Batch size = 64


Overall accuracy is around 91%; precision, recall, and F1 are computed for the minority class (metaphorical usages); when a metaphorical usage is predicted, it is correct about 3/4 of the times (precision) - and we are able to predict about half of the metaphorical usages that are present in the dataset (recall).

In [ ]:
pred.metrics

{'test_accuracy': 0.9137452711223203,
 'test_f1': 0.6208425720620843,
 'test_loss': 0.2568821609020233,
 'test_precision': 0.775623268698061,
 'test_recall': 0.5175600739371534,
 'test_runtime': 31.8914,
 'test_samples_per_second': 124.328,
 'test_steps_per_second': 1.944}

Print 100 sentences, together with predict label and gold label

In [ ]:
list(zip(sentences_test[0:100], positions_test[0:100], np.argmax(pred.predictions[0:100], axis=1), labels_test[0:100]))

[('Design : Crossed lines over the toytown tram : City transport could soon be back on the right track , says Jonathan Glancey',
  3,
  0,
  1),
 ('Design : Crossed lines over the toytown tram : City transport could soon be back on the right track , says Jonathan Glancey',
  7,
  0,
  0),
 ('Design : Crossed lines over the toytown tram : City transport could soon be back on the right track , says Jonathan Glancey',
  19,
  0,
  0),
 ('MODERN trams , as most continental Europeans know , neither shake nor rattle , nor do they roll .',
  5,
  0,
  0),
 ('MODERN trams , as most continental Europeans know , neither shake nor rattle , nor do they roll .',
  6,
  0,
  0),
 ('MODERN trams , as most continental Europeans know , neither shake nor rattle , nor do they roll .',
  13,
  0,
  0),
 ('Sleek , solidly built , gentle on the environment , they are often an ideal form of city transport .',
  1,
  0,
  1),
 ('Sleek , solidly built , gentle on the environment , they are often an ideal form 